In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging

import pandas as pd

import dataflow.system as dtfsys
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.client as icdcl
import im_v2.crypto_chassis.data.client as iccdc
import market_data.market_data_example as mdmadaex

/app/amp/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
log_level = logging.INFO
hdbg.init_logger(verbosity=log_level)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-ee541b40-807f-4947-9565-ef21754bce6e.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.2' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4372_fix_and_test_gallery_market_data_notebook'
  hash='3a07e346a'
  # Last commits:
    *   3a07e346a Grigorii Pomazkin Merge branch 'master' into CmTask4372_fix_and_test_gallery_market_data_notebook (36 minutes ago) Wed May 24 12:36:05 2023  (HEAD -> CmTask4372_fix_and_test_gallery_market_data_notebook, origin/CmTask4372_fix_and_test_gallery_market_data_notebook)
    |\  
    | * e432ad0ad Sonya Nikiforova CmTask4380

# OHLCV market data

In [4]:
# Initialize the client.
data_version = "v2"
universe_version = "v4"
dataset = "ohlcv"
contract_type = "futures"
data_snapshot = "20220620"
im_client = icdcl.get_CcxtHistoricalPqByTileClient_example1(
    data_version,
    universe_version,
    dataset,
    contract_type,
    data_snapshot=data_snapshot,
)
# Set expected values.
full_symbols = ["binance::BTC_USDT", "binance::ADA_USDT"]
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-01 13:05:00+00:00")
columns = None
filter_data_mode = "assert"
actual_df = im_client.read_data(
    full_symbols, start_ts, end_ts, columns, filter_data_mode
)

In [5]:
symbols = im_client.get_universe()[3:6]
display(symbols)

['binance::BTC_USDT', 'binance::EOS_USDT', 'binance::ETH_USDT']

In [6]:
asset_ids = im_client.get_asset_ids_from_full_symbols(symbols)
print(asset_ids)

[1467591036, 2061507978, 1464553467]


In [7]:
columns = None
column_remap = None
market_data = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client, asset_ids, columns, column_remap
)

In [8]:
# TODO(*): Document whether we need UTC.
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-02 13:05:00+00:00")
ts_col_name = "knowledge_timestamp"
df = market_data.get_data_for_interval(
    start_ts,
    end_ts,
    ts_col_name,
    asset_ids,
)
hpandas.df_to_str(df, log_level=log_level)

,asset_id,full_symbol,open,high,low,close,volume,knowledge_timestamp,start_ts
2022-05-01 09:00:00-04:00,1464553467,binance::ETH_USDT,2760.0,2762.56,2759.13,2762.56,287.454,2022-06-24 11:10:10.287766+00:00,2022-05-01 08:59:00-04:00
2022-05-01 09:00:00-04:00,1467591036,binance::BTC_USDT,37935.7,37959.2,37930.7,37959.2,71.386,2022-06-24 05:47:16.075108+00:00,2022-05-01 08:59:00-04:00
2022-05-01 09:00:00-04:00,2061507978,binance::EOS_USDT,2.043,2.045,2.043,2.045,10408.8,2022-06-24 09:24:17.040013+00:00,2022-05-01 08:59:00-04:00
,...,...,...,...,...,...,...,...,...
2022-05-02 09:04:00-04:00,1464553467,binance::ETH_USDT,2794.29,2797.24,2794.29,2796.12,290.666,2022-06-24 11:10:10.287766+00:00,2022-05-02 09:03:00-04:00
2022-05-02 09:04:00-04:00,1467591036,binance::BTC_USDT,38505.1,38540.0,38505.0,38529.0,79.599,2022-06-24 05:47:16.075108+00:00,2022-05-02 09:03:00-04:00
2022-05-02 09:04:00-04:00,2061507978,binance::EOS_USDT,2.055,2.062,2.054,2.06,88356.6,2022-06-24 09:24:17.040013+00:00,2022-05-02 09:03:00-04:00


In [9]:
data_source_node = dtfsys.HistoricalDataSource(
    "load_prices",
    market_data,
    "end_ts",
    True,
    col_names_to_remove=["knowledge_timestamp", "start_ts", "full_symbol"],
)
data_source_node.set_fit_intervals([(start_ts, end_ts)])

In [10]:
data = data_source_node.fit()["df_out"]
hpandas.df_to_str(data, log_level=log_level)

close                             high                              low                             open                           volume                      
                          1464553467 1467591036 2061507978 1464553467 1467591036 2061507978 1464553467 1467591036 2061507978 1464553467 1467591036 2061507978 1464553467 1467591036 2061507978
2022-05-01 09:00:00-04:00    2762.56    37959.2      2.045    2762.56    37959.2      2.045    2759.13    37930.7      2.043     2760.0    37935.7      2.043    287.454     71.386    10408.8
2022-05-01 09:01:00-04:00    2764.73    37973.9      2.049     2765.3    37987.8       2.05    2762.55    37959.1      2.044    2762.55    37959.2      2.045    1008.01    146.115    66058.2
2022-05-01 09:02:00-04:00    2763.97    37957.5      2.047    2764.73    37974.0      2.049    2763.09    37953.3      2.046    2764.73    37973.9      2.048    583.906     78.208    20788.0
                                 ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...
2022-05-02 09:03:00-04:00    2794.29    38505.0      2.055    2796.36    38518.9      2.055    2793.79    38500.0      2.049    2795.95    38518.9      2.051    611.852     85.365   120724.6
2022-05-02 09:04:00-04:00    2796.12    38529.0       2.06    2797.24    38540.0      2.062    2794.29    38505.0      2.054    2794.29    38505.1      2.055    290.666     79.599    88356.6
2022-05-02 09:05:00-04:00    2797.04    38538.1      2.061    2797.66    38540.9      2.061    2796.11    38528.0      2.059    2796.11    38529.0      2.061    240.935     57.524    34581.6

# Stitched OHLCV and bid/ask data

In [11]:
universe_version = "v4"
contract_type = "futures"
data_snapshot = "20220707"
#
dataset1 = "ohlcv"
im_client1 = iccdc.get_CryptoChassisHistoricalPqByTileClient_example1(
    universe_version,
    dataset1,
    contract_type,
    data_snapshot,
)
#
dataset2 = "bid_ask"
im_client2 = iccdc.get_CryptoChassisHistoricalPqByTileClient_example1(
    universe_version,
    dataset2,
    contract_type,
    data_snapshot,
)
#
asset_ids = [1467591036, 1464553467]
columns = None
column_remap = None
wall_clock_time = None
filter_data_mode = "assert"
#
im_client_market_data1 = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client1,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)
im_client_market_data2 = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client2,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)
market_data = mdmadaex.get_HorizontalStitchedMarketData_example1(
    im_client_market_data1,
    im_client_market_data2,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)

In [12]:
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-02 13:05:00+00:00")
ts_col_name = "knowledge_timestamp"
df = market_data.get_data_for_interval(
    start_ts,
    end_ts,
    ts_col_name,
    asset_ids,
)
hpandas.df_to_str(df, log_level=log_level)

,asset_id,full_symbol,open,high,low,close,volume,vwap,number_of_trades,twap,knowledge_timestamp,start_ts,bid_price,bid_size,ask_price,ask_size
2022-05-01 09:00:00-04:00,1464553467,binance::ETH_USDT,2762.55,2765.3,2762.55,2764.73,1008.01,2764.2581,490.0,2764.1087,2022-06-20 09:49:40.140622+00:00,2022-05-01 08:59:00-04:00,2764.152123,2436.613,2764.003005,909.711
2022-05-01 09:00:00-04:00,1467591036,binance::BTC_USDT,37959.2,37987.8,37959.1,37973.9,146.115,37974.469,694.0,37974.598,2022-06-20 09:48:46.910826+00:00,2022-05-01 08:59:00-04:00,37973.770035,228.977,37974.633461,175.528
2022-05-01 09:01:00-04:00,1464553467,binance::ETH_USDT,2764.73,2764.73,2763.09,2763.97,584.206,2763.9137,302.0,2763.8355,2022-06-20 09:49:40.140622+00:00,2022-05-01 09:00:00-04:00,2763.800863,2165.185,2763.885699,862.162
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-02 09:03:00-04:00,1467591036,binance::BTC_USDT,38505.1,38540.0,38505.0,38529.0,79.599,38525.57,571.0,38525.125,2022-06-20 09:48:46.910826+00:00,2022-05-02 09:02:00-04:00,38527.300859,344.537,38523.1902,160.702
2022-05-02 09:04:00-04:00,1464553467,binance::ETH_USDT,2796.11,2797.66,2796.11,2797.04,240.935,2797.0636,231.0,2796.899,2022-06-20 09:49:40.140622+00:00,2022-05-02 09:03:00-04:00,2796.875836,1206.289,2796.964358,1318.165
2022-05-02 09:04:00-04:00,1467591036,binance::BTC_USDT,38529.0,38540.9,38528.0,38538.1,57.524,38533.817,474.0,38533.746,2022-06-20 09:48:46.910826+00:00,2022-05-02 09:03:00-04:00,38532.121083,252.337,38535.484828,235.788


In [13]:
data_source_node = dtfsys.HistoricalDataSource(
    "load_prices",
    market_data,
    "end_ts",
    True,
    col_names_to_remove=["knowledge_timestamp", "start_ts", "full_symbol"],
)
data_source_node.set_fit_intervals([(start_ts, end_ts)])

In [14]:
data = data_source_node.fit()["df_out"]
hpandas.df_to_str(data, log_level=log_level)

ask_price                 ask_size               bid_price                 bid_size                 close                  high                   low            number_of_trades                  open                  twap                volume                  vwap           
                            1464553467    1467591036 1464553467 1467591036   1464553467    1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036       1464553467 1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036
2022-05-01 09:00:00-04:00  2764.003005  37974.633461    909.711    175.528  2764.152123  37973.770035   2436.613    228.977    2764.73    37973.9     2765.3    37987.8    2762.55    37959.1            490.0      694.0    2762.55    37959.2  2764.1087  37974.598    1008.01    146.115  2764.2581  37974.469
2022-05-01 09:01:00-04:00  2763.885699   37963.92703    862.162    177.092  2763.800863  37962.984408   2165.185    191.785    2763.97    37957.5    2764.73    37974.0    2763.09    37953.3            302.0      471.0    2764.73    37973.9  2763.8355  37962.282    584.206     78.208  2763.9137   37964.79
2022-05-01 09:02:00-04:00  2763.430308  37957.112679     864.03    116.318  2763.207318  37955.899564   1951.813     169.22    2762.42    37954.0    2763.98    37957.6    2762.36    37945.6            265.0      328.0    2763.97    37957.6  2763.3025  37954.937    466.446     44.477  2763.2716  37954.824
                                   ...           ...        ...        ...          ...           ...        ...        ...        ...        ...        ...        ...        ...        ...              ...        ...        ...        ...        ...        ...        ...        ...        ...        ...
2022-05-02 09:03:00-04:00  2796.219035    38523.1902   1543.286    160.702  2795.911376  38527.300859   1344.882    344.537    2796.12    38529.0    2797.24    38540.0    2794.29    38505.0            334.0      571.0    2794.29    38505.1  2795.9478  38525.125    290.666     79.599  2795.8491   38525.57
2022-05-02 09:04:00-04:00  2796.964358  38535.484828   1318.165    235.788  2796.875836  38532.121083   1206.289    252.337    2797.04    38538.1    2797.66    38540.9    2796.11    38528.0            231.0      474.0    2796.11    38529.0   2796.899  38533.746    240.935     57.524  2797.0636  38533.817
2022-05-02 09:05:00-04:00  2798.602739  38547.402691     940.77     188.32  2798.497657  38545.895797   1281.511    297.967    2799.29    38550.0    2799.42    38554.4    2797.05    38534.0            585.0      625.0    2797.05    38538.1  2798.6345   38547.26   1046.668     110.09  2798.6133  38548.878